神经网络模型一般依靠随机梯度下降进行模型训练和参数更新，网络的最终性能与收敛得到的最优解直接相关，而收敛效果实际上有很大程度取决于网络参数的初始化。糟糕的初始化不仅会影响网络收敛，甚至会导致"梯度弥散"或"梯度爆炸"。

### 全零初始化
通过合理的数据预处理和规范化，当网络收敛到稳定状态时，参数（权值）在理想情况下应基本保持正负各半的状态（此时期望为0）。因此，一种简单且听起来合理的参数初始化做法是，将所有参数都初始化为0，因为这样可使得初始化全零时参数的期望与网络稳定时参数的期望一致为零。但是参数全为0时网络不同神经元的输出必然相同，相同输出则导致梯度更新完全一样，这样便会令更新后的参数仍然保持一样的状态。换句话说，如若参数进行了全零初始化，那么网络神经元将无法更新参数，从而无法进行模型训练。

### 随机初始化
将参数值随机设定为接近于0的一个很小的随机数（有正有负）。在实际应用中，随机参数服从高斯分布或均匀分布都是较有效的初始化方式。